# Sources of levels and lines: Uncertainties

---

Currently, Carsus can handle two different sources of energy levels and transition lines: the Robert Kurucz's line list (GFALL) and the Chianti Database. 

But, what are the effects of choosing a specific source of levels and lines on the TARDIS spectra? To gain some insight, we ran TARDIS simulations of a Blondin model with abundances of `Si`, `S`, `Ca`, `Fe`, `Co`, and `Ni`: 

- A "baseline" simulation using only data from the Kurucz line list.
- Individual simulations in [opacity machine](#) mode using data from the Kurucz line list, plus switching one specie from the Chianti Database (see [Appendix I](#appendix)). 

Then, we added all the differences in luminosity between the baseline model and the Chianti-based spectra (**residuals**) and look for the most affected regions of the spectrum:

<img src="img/S_20d.png">

<img src="img/Fe_20d.png">

Luminosities can be affected up to 50% for some wavelengths, just for using different atomic data sources!.

<a id='appendix_i'></a>

# Appendix I: How to reproduce the atomic files

First, download and extract the Chianti Database and set the `XUVTOP` environment variable.

```
cd $HOME/Downloads
wget http://www.chiantidatabase.org/download/CHIANTI_9.0.1_data.tar.gz
mkdir chianti
tar -xvzf CHIANTI_9.0.1_data.tar.gz -C chianti
```

In [ ]:
import os

os.environ['XUVTOP'] = os.path.expanduser('~/Downloads/chianti')

Then, import the following classes and define the `create_atom_data` function.

In [ ]:
from carsus.io.nist import NISTWeightsComp, NISTIonizationEnergies
from carsus.io.kurucz import GFALLReader
from carsus.io.zeta import KnoxLongZeta
from carsus.io.chianti_ import ChiantiReader
from carsus.io.output import TARDISAtomData

In [ ]:
def create_atom_data(gfall_ions, chianti_ions, output_dir):

    parsers = []
    parsers.append(NISTWeightsComp())
    parsers.append(NISTIonizationEnergies(gfall_ions))
    parsers.append(GFALLReader(gfall_ions))
    parsers.append(KnoxLongZeta())

    if chianti_ions:
        parsers.append(ChiantiReader(chianti_ions, priority=20))
        ions_string = chianti_ions.replace(' ', '_').replace(';', '')
        output_file = 'kurucz_cd23_latest_chianti_{}.h5'.format(ions_string)

    else:
        output_file = 'kurucz_cd23_latest.h5'

    os.makedirs(output_dir, exist_ok=True)
    fname = os.path.join(output_dir, output_file)
    
    atom_data = TARDISAtomData(*parsers)
    atom_data.to_hdf(fname)

    return

Finally, to create an atomic file with all available species from GFALL except for $\text{Si II}$ just run:

```python
create_atom_data('H-Zn', 'Si 1', os.path.expanduser('~/Downloads/tardis-data'))
```

and repeat for every `Si`, `S`, `Ca`, `Fe` and `Ni` ion [available in the Chianti Database](https://www.chiantidatabase.org/chianti_direct_data.html).

# Appendix II: Run TARDIS as an opacity machine

<div class="alert alert-warning">

**NOTE:** It's necessary to pickle the model using the parameter `protocol=0`, otherwise load will fail.

</div>

```python
import pickle 

with open('toy06_20d_model.pickle', 'rb') as pm:
    baseline_model = pickle.load(pm)
```

```python
from tardis.io.atom_data import AtomData
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation

tardis_config = Configuration.from_yaml('blondin_model_compare_06.yml')
tardis_config['montecarlo']['iterations'] = 1
atom_data = AtomData.from_hdf('kurucz_cd23_latest_Si_1.h5')   
simulation = Simulation.from_config(tardis_config, atom_data=atom_data, 
                                        model=baseline_model)
simulation.run()
```